In [81]:
import os
import csv
import shutil
import tarfile
import numpy as np
import pandas as pd
from urllib.request import urlretrieve
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

In [82]:
data=pd.read_csv('data.csv')

In [83]:
data.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5


In [84]:
data = data[np.isfinite(data['shot_made_flag'])]
data['action_type']=data['action_type'].astype('category')

In [85]:
features = data[['seconds_remaining','minutes_remaining',
                 'combined_shot_type','period','playoffs','season',
                 'shot_distance','shot_zone_area','game_date']]
#features['total_seconds_remaining'] = 60 * features['minutes_remaining'] + features['seconds_remaining']
#features.drop(['seconds_remaining','minutes_remaining'],inplace=True, axis=1)
features.head()

,seconds_remaining,minutes_remaining,combined_shot_type,period,playoffs,season,shot_distance,shot_zone_area,game_date
1,22,10,Jump Shot,1,0,2000-01,15,Left Side(L),2000-10-31
2,45,7,Jump Shot,1,0,2000-01,16,Left Side Center(LC),2000-10-31
3,52,6,Jump Shot,1,0,2000-01,22,Right Side Center(RC),2000-10-31
4,19,6,Dunk,2,0,2000-01,0,Center(C),2000-10-31
5,32,9,Jump Shot,3,0,2000-01,14,Left Side(L),2000-10-31


In [86]:
le = LabelEncoder()
for column in features.columns:
    print(column)
    if features[column].dtype == type(object):
        features[column] = le.fit_transform(features[column])
features.head()

seconds_remaining
minutes_remaining
combined_shot_type
period
playoffs
season
shot_distance
shot_zone_area
game_date


/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,seconds_remaining,minutes_remaining,combined_shot_type,period,playoffs,season,shot_distance,shot_zone_area,game_date
1,22,10,3,1,0,4,15,3,309
2,45,7,3,1,0,4,16,2,309
3,52,6,3,1,0,4,22,4,309
4,19,6,1,2,0,4,0,1,309
5,32,9,3,3,0,4,14,3,309


In [87]:
target = data['shot_made_flag']
target.head()

1    0.0
2    1.0
3    0.0
4    1.0
5    0.0
Name: shot_made_flag, dtype: float64

In [88]:
X_train, X_test, y_train, y_test = train_test_split(features, target)

In [91]:
#Decision Tree
max_leaf_nodes = X_train.shape[0]
max_leaf_nodes = 10

dt = DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes,
                            criterion='entropy')
dt.fit(X_train, y_train);

n_nodes = dt.tree_.node_count

predicted = dt.predict(X_test)
print(f"Accuracy: {accuracy_score(predicted, y_test):.3f}")

Accuracy: 0.615


In [93]:
# Naive Bayes
NB = MultinomialNB()
NB.fit(X_train, y_train)

NB_prediction = NB.predict(X_test)
print(f"Accuracy: {accuracy_score(NB_prediction, y_test):.3f}")

Accuracy: 0.588


In [94]:
#kNN Classifer
KNN = KNeighborsClassifier(n_neighbors=3)
KNN.fit(X_train, y_train)

KNN_prediction = KNN.predict(X_test)
print(f"Accuracy: {accuracy_score(KNN_prediction, y_test):.3f}")

Accuracy: 0.538
